In [50]:
import psycopg2
import contractions
import string
import re
import pandas as pd
import numpy as np
import networkx as nx
import nltk
from nltk.util import ngrams
import tqdm

In [51]:
def generate_ngrams(text, x, y):
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    
    # Generate n-grams for n ranging from x to y
    ngrams_list = []
    for n in range(x, y+1):
        ngrams_ = ngrams(words, n)
        ngrams_ = [' '.join(grams) for grams in ngrams_]
        ngrams_list.extend(ngrams_)
    
    return ngrams_list

def generate_ngrams_listofsentences(list_of_sentences, x, y):
    ngrams_list = []
    for sentence in list_of_sentences:
        ngrams_list.extend(generate_ngrams(sentence, x, y))
    return ngrams_list

In [52]:
def preprocess_heading(text):
    # Lowercase the text
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove contractions
    text = contractions.fix(text)
    return text

def preprocess_paragraph(text):
    # split into lines
    lines = text.split('\n')
    # remove leading and trailing spaces on each line
    lines = [line.strip() for line in lines]
    # remove empty lines
    lines = [line for line in lines if len(line) > 0]
    # Lowercase the text
    lines = [line.lower() for line in lines]
    # Remove punctuation
    lines = [line.translate(str.maketrans('', '', string.punctuation)) for line in lines]
    # Remove contractions
    lines = [contractions.fix(line) for line in lines]
    return lines

def preprocess_text(text):
    # if more than 4 words, return empty string
    if len(text.split()) > 4:
        return ''
    # Lowercase the text
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove contractions
    text = contractions.fix(text)
    return text

In [53]:
# connect to database
conn = psycopg2.connect(
    host="localhost",
    port=5450,
    database="postgres",
    user="postgres",
    password="postgres"
)

# create cursor
cur = conn.cursor()

In [54]:
# get the topics table, select random 20 rows
cur.execute("SELECT * FROM topics")
topics = cur.fetchall()

# get the columns
cols = [desc[0] for desc in cur.description]

# create dataframe
topics_df = pd.DataFrame(topics, columns=cols)

topics_df.head(20)

,topic,topics_preprocessed
0,communication,communication
1,energy,energy
2,and water supply. however,and water supply however
3,the demand for infrastructure services increas...,
4,extreme weather events,extreme weather events
5,and other climate-related phenomena pose signi...,
6,there is a need to address issues of equity an...,
7,particularly in developing countries,particularly in developing countries
8,lack basic infrastructure services. rethinking...,
9,the world needs to rethink infrastructure acce...,


In [55]:
# preprocess the topics and add it as a new column
topics_df['topics_preprocessed'] = topics_df['topic'].apply(preprocess_text)

topics_df.head(20)

,topic,topics_preprocessed
0,communication,communication
1,energy,energy
2,and water supply. however,and water supply however
3,the demand for infrastructure services increas...,
4,extreme weather events,extreme weather events
5,and other climate-related phenomena pose signi...,
6,there is a need to address issues of equity an...,
7,particularly in developing countries,particularly in developing countries
8,lack basic infrastructure services. rethinking...,
9,the world needs to rethink infrastructure acce...,


In [56]:
# print total number of topics
print('Total number of topics: ', len(topics_df))
# print number of topics with non empty preprocessed topics
print('Number of topics with non empty preprocessed topics: ', len(topics_df[topics_df['topics_preprocessed'] != '']))
# print the number of unique topics based on the preprocessed topics
print('Number of unique topics: ', len(topics_df['topics_preprocessed'].unique()))

Total number of topics:  67750
Number of topics with non empty preprocessed topics:  59736
Number of unique topics:  59522


In [57]:
# import data into pandas dataframe
cur.execute("SELECT * FROM resources")

# get column names
colnames = [desc[0] for desc in cur.description]

# create dataframe
df = pd.DataFrame(cur.fetchall(), columns=colnames)

In [58]:
# preproces the heading and content and ad them as new columns
df['heading_preprocessed'] = df['heading'].apply(preprocess_heading)
df['content_preprocessed'] = df['content'].apply(preprocess_paragraph)

df.head()

,url,hid,heading,hnum,next_heading,subheading,content,topics,heading_preprocessed,content_preprocessed,heading_ngrams,content_ngrams,id
0,https://adaptive.live/blog/rethink-infrastruct...,h2,The Increasing Data Breach Problem,1,NaN,2.0,The problem of cyber breaches is only getting ...,"[""data breach problem"", ""cyber breaches"", ""tre...",the increasing data breach problem,[the problem of cyber breaches is only getting...,None,None,2
1,https://adaptive.live/blog/rethink-infrastruct...,h2,Insider Threat: a new problem within organizat...,2,NaN,3.0,"Recently, a new type of threat has started imp...","[""insider threats"", ""cyber security risk"", ""or...",insider threat a new problem within organizations,[recently a new type of threat has started imp...,None,None,3
2,https://adaptive.live/blog/rethink-infrastruct...,h2,Complexity of the Modern-day Infrastructure,3,4.0,8.0,In order to understand the problems with infra...,"[""complexity"", ""modern-day infrastructure"", ""p...",complexity of the modernday infrastructure,[in order to understand the problems with infr...,None,None,4
3,https://adaptive.live/blog/rethink-infrastruct...,h3,Remote-first trend post-covid,4,NaN,5.0,"In March 2020 Covid struck, and remote work we...","[""remote work"", ""covid"", ""mainstream"", ""rushed...",remotefirst trend postcovid,[in march 2020 covid struck and remote work we...,None,None,5
4,https://adaptive.live/blog/rethink-infrastruct...,h3,Increased Cloud Adoption,5,NaN,6.0,A common infrastructure setup that companies u...,"[""cloud adoption"", ""infrastructure setup"", ""on...",increased cloud adoption,[a common infrastructure setup that companies ...,None,None,6


In [59]:
# generate ngrams for heading rangint (1-4) and add it as a new column
df['heading_ngrams'] = df['heading_preprocessed'].apply(generate_ngrams, args=(1,4))

df.head()

,url,hid,heading,hnum,next_heading,subheading,content,topics,heading_preprocessed,content_preprocessed,heading_ngrams,content_ngrams,id
0,https://adaptive.live/blog/rethink-infrastruct...,h2,The Increasing Data Breach Problem,1,NaN,2.0,The problem of cyber breaches is only getting ...,"[""data breach problem"", ""cyber breaches"", ""tre...",the increasing data breach problem,[the problem of cyber breaches is only getting...,"[the, increasing, data, breach, problem, the i...",None,2
1,https://adaptive.live/blog/rethink-infrastruct...,h2,Insider Threat: a new problem within organizat...,2,NaN,3.0,"Recently, a new type of threat has started imp...","[""insider threats"", ""cyber security risk"", ""or...",insider threat a new problem within organizations,[recently a new type of threat has started imp...,"[insider, threat, a, new, problem, within, org...",None,3
2,https://adaptive.live/blog/rethink-infrastruct...,h2,Complexity of the Modern-day Infrastructure,3,4.0,8.0,In order to understand the problems with infra...,"[""complexity"", ""modern-day infrastructure"", ""p...",complexity of the modernday infrastructure,[in order to understand the problems with infr...,"[complexity, of, the, modernday, infrastructur...",None,4
3,https://adaptive.live/blog/rethink-infrastruct...,h3,Remote-first trend post-covid,4,NaN,5.0,"In March 2020 Covid struck, and remote work we...","[""remote work"", ""covid"", ""mainstream"", ""rushed...",remotefirst trend postcovid,[in march 2020 covid struck and remote work we...,"[remotefirst, trend, postcovid, remotefirst tr...",None,5
4,https://adaptive.live/blog/rethink-infrastruct...,h3,Increased Cloud Adoption,5,NaN,6.0,A common infrastructure setup that companies u...,"[""cloud adoption"", ""infrastructure setup"", ""on...",increased cloud adoption,[a common infrastructure setup that companies ...,"[increased, cloud, adoption, increased cloud, ...",None,6


In [60]:
# generate ngrams for content ranging (1-4) and add it as a new column
df['content_ngrams'] = df['content_preprocessed'].apply(generate_ngrams_listofsentences, args=(1,4))

df.head()

,url,hid,heading,hnum,next_heading,subheading,content,topics,heading_preprocessed,content_preprocessed,heading_ngrams,content_ngrams,id
0,https://adaptive.live/blog/rethink-infrastruct...,h2,The Increasing Data Breach Problem,1,NaN,2.0,The problem of cyber breaches is only getting ...,"[""data breach problem"", ""cyber breaches"", ""tre...",the increasing data breach problem,[the problem of cyber breaches is only getting...,"[the, increasing, data, breach, problem, the i...","[the, problem, of, cyber, breaches, is, only, ...",2
1,https://adaptive.live/blog/rethink-infrastruct...,h2,Insider Threat: a new problem within organizat...,2,NaN,3.0,"Recently, a new type of threat has started imp...","[""insider threats"", ""cyber security risk"", ""or...",insider threat a new problem within organizations,[recently a new type of threat has started imp...,"[insider, threat, a, new, problem, within, org...","[recently, a, new, type, of, threat, has, star...",3
2,https://adaptive.live/blog/rethink-infrastruct...,h2,Complexity of the Modern-day Infrastructure,3,4.0,8.0,In order to understand the problems with infra...,"[""complexity"", ""modern-day infrastructure"", ""p...",complexity of the modernday infrastructure,[in order to understand the problems with infr...,"[complexity, of, the, modernday, infrastructur...","[in, order, to, understand, the, problems, wit...",4
3,https://adaptive.live/blog/rethink-infrastruct...,h3,Remote-first trend post-covid,4,NaN,5.0,"In March 2020 Covid struck, and remote work we...","[""remote work"", ""covid"", ""mainstream"", ""rushed...",remotefirst trend postcovid,[in march 2020 covid struck and remote work we...,"[remotefirst, trend, postcovid, remotefirst tr...","[in, march, 2020, covid, struck, and, remote, ...",5
4,https://adaptive.live/blog/rethink-infrastruct...,h3,Increased Cloud Adoption,5,NaN,6.0,A common infrastructure setup that companies u...,"[""cloud adoption"", ""infrastructure setup"", ""on...",increased cloud adoption,[a common infrastructure setup that companies ...,"[increased, cloud, adoption, increased cloud, ...","[a, common, infrastructure, setup, that, compa...",6


In [61]:
# find the topic row from the database where the topic is energy, rollback any transaction that is in progress
conn.rollback()
cur.execute("SELECT * FROM topics WHERE topic = 'energy'")
topic = cur.fetchone()
print(topic)

('energy', 'energy')


In [62]:
# update the topics table with the preprocessed topics, use tqdm to show progress bar
for index, row in tqdm.tqdm(topics_df.iterrows()):
    cur.execute("UPDATE topics SET topics_preprocessed = %s WHERE topic = %s", (row['topics_preprocessed'], row['topic']))
    conn.commit()

67750it [00:34, 1937.82it/s]


In [47]:
# add a new column to the resources table with SERIAL PRIMARY KEY
cur.execute("ALTER TABLE resources ADD COLUMN id SERIAL PRIMARY KEY")
conn.commit()

In [ ]:
# for each row in the resources table where id = row id, update the heading and content columns with the preprocessed heading and content
for index, row in tqdm.tqdm(df.iterrows()):
    # update heading_preprocessed column
    print(row['id'])
    cur.execute("UPDATE resources SET heading_preprocessed = %s WHERE id = %s", (row['heading_preprocessed'], row['id']))
    # update content_preprocessed column
    cur.execute("UPDATE resources SET content_preprocessed = %s WHERE id = %s", (row['content_preprocessed'], row['id']))
    # update heading_ngrams column
    cur.execute("UPDATE resources SET heading_ngrams = %s WHERE id = %s", (row['heading_ngrams'], row['id']))
    # update content_ngrams column
    cur.execute("UPDATE resources SET content_ngrams = %s WHERE id = %s", (row['content_ngrams'], row['id']))
    conn.commit()

# close the cursor and connection
cur.close()
conn.close()

In [49]:
# copy the resources table into a new resourcesbackup table
cur.execute("CREATE TABLE resourcesbackup AS TABLE resources")
conn.commit()